In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os

from importlib import reload
from neuropacks import AC1, PVC11, RET2
from noise_correlations import analysis, plot, utils

%matplotlib inline

In [2]:
base_path = '/Volumes/pss'

# PVC11: Monkey, primary visual cortex, Kohn Lab

In [3]:
# PVC11: Monkey, primary visual cortex, Kohn Lab
pvc11_paths = [
    os.path.join(base_path,
                 f'data/pvc11/data/spikes_gratings/data_monkey{idx}_gratings.mat')
    for idx in range(1, 4)
]
# Get design matrices
pvc11_packs = [PVC11(pvc11_path) for pvc11_path in pvc11_paths]
pvc11_Xs = [pack.get_response_matrix(transform=None) for pack in pvc11_packs]
pvc11_n_units = [X.shape[1] for X in pvc11_Xs]
pvc11_n_samples = [X.shape[0] for X in pvc11_Xs]
# Get stimuli
stimuli = pvc11_packs[0].get_design_matrix(form='angle')

In [4]:
var_to_mean_threshold = 10
responsive_threshold = 2. * 1.28
modulation_frac = 0.60

In [5]:
pvc11_non_responsive_for_stim = [
    utils.get_nonresponsive_for_stim(X, stimuli)
    for X in pvc11_Xs
]

for idx, non_responsive in enumerate(pvc11_non_responsive_for_stim):
    if non_responsive.size > 0:
        pvc11_Xs[idx] = np.delete(pvc11_Xs[idx], non_responsive, axis=1)

In [6]:
pvc11_responsive_units = [
    utils.get_responsive_units(
        X=X, stimuli=stimuli,
        aggregator=np.mean,
        peak_response=responsive_threshold,
        variance_to_mean=var_to_mean_threshold)
    for X in pvc11_Xs
]
pvc11_n_responsive_units = [responsive.sum() for responsive in pvc11_responsive_units]

pvc11_tuned_units = [
    utils.get_tuned_units(
        X=X, stimuli=stimuli,
        aggregator=np.mean,
        peak_response=responsive_threshold,
        tuning_criteria='modulation_frac',
        modulation_frac=modulation_frac,
        variance_to_mean=var_to_mean_threshold)
    for X in pvc11_Xs
]
pvc11_n_tuned_units = [tuned.sum() for tuned in pvc11_tuned_units]

In [7]:
print('Number of units:', pvc11_n_units)
print('Number of responsive units:', pvc11_n_responsive_units)
print('Number of tuned units:', pvc11_n_tuned_units)

Number of units: [106, 88, 112]
Number of responsive units: [88, 60, 108]
Number of tuned units: [71, 55, 90]


# AC1: Rat, $\mu$ECoG, Bouchard Lab

In [8]:
data_path = os.path.join(base_path, 'data/ac1/R32_B7_HG_ext_rsp.h5')
ac = AC1(data_path=data_path)

In [9]:
ac_X = ac.get_response_matrix(amplitudes=[2, 3, 4, 5, 6, 7])
stimuli = np.log(ac.get_design_matrix(amplitudes=[2, 3, 4, 5, 6, 7]))
unique_stimuli = np.unique(stimuli)

In [10]:
var_to_mean_thres = 10
responsive_thres = 1
modulation_frac_thres = 0.75

In [11]:
responsive_units = utils.get_responsive_units(
    X=ac_X, stimuli=stimuli,
    aggregator=np.mean,
    peak_response=responsive_thres,
    variance_to_mean=var_to_mean_thres)
tuned_units = utils.get_tuned_units(
    X=ac_X, stimuli=stimuli,
    aggregator=np.mean,
    peak_response=responsive_thres,
    tuning_criteria='modulation_frac',
    modulation_frac=modulation_frac_thres,
    variance_to_mean=var_to_mean_thres)

In [12]:
print('Number of units:', ac_X.shape[1])
print('Number of responsive units:', responsive_units.sum())
print('Number of tuned units:', tuned_units.sum())

Number of units: 60
Number of responsive units: 58
Number of tuned units: 57


# RET2: Mouse, retinal ganglion cells, Feller Lab

In [13]:
ret2_path = os.path.join(base_path, 'data/ret2/200114_fov1_data.mat')
ret2 = RET2(data_path=ret2_path)

In [14]:
ret2_X = ret2.get_response_matrix(cells='all', response='max')
stimuli = ret2.angles
unique_stimuli = ret2.unique_angles

In [15]:
responsive_threshold = 1.0

In [16]:
tuned_units = np.zeros(ret2_X.shape[1]).astype(bool)
tuned_units[ret2.tuned_cells] = True
responsive_units = utils.get_responsive_units(
    X=ret2_X, stimuli=stimuli,
    aggregator=np.mean,
    peak_response=responsive_threshold)

In [17]:
print('Number of units:', ret2_X.shape[1])
print('Number of responsive units:', responsive_units.sum())
print('Number of tuned units:', tuned_units.sum())

Number of units: 832
Number of responsive units: 368
Number of tuned units: 54
